In [2]:
# Importa as bibliotecas necessárias
%run '~/git/Bibliotecas.ipynb'

# Importação

In [3]:
# Carrega a base de dados contendo os registros de importação de fertilizantes
im=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/i.csv').drop(columns='date')
ch=pd.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/c.csv').drop(columns='date')
q=pq.read_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv')

NameError: name 'pq' is not defined

In [4]:
r=q

NameError: name 'q' is not defined

In [4]:
r=round(q.groupby('CO_ANO')[['erro_mm','erro_s_mm']].mean()/10**6).reset_index()
r['dif']=r.erro_s_mm-r.erro_mm
r.sort_values(by='dif', ascending=False)

,CO_ANO,erro_mm,erro_s_mm,dif
13,2020,29.0,37.0,8.0
9,2016,25.0,31.0,6.0
16,2023,44.0,49.0,5.0
5,2012,31.0,36.0,5.0
14,2021,51.0,54.0,3.0
8,2015,28.0,31.0,3.0
11,2018,54.0,56.0,2.0
6,2013,24.0,26.0,2.0
4,2011,24.0,24.0,0.0
7,2014,26.0,26.0,0.0


In [5]:
r=round(q.groupby('CO_MES')[['erro_mm','erro_s_mm']].mean()/10**6).reset_index()
r['dif']=r.erro_s_mm-r.erro_mm
r.sort_values(by='dif', ascending=False)

,CO_MES,erro_mm,erro_s_mm,dif
1,2,33.0,39.0,6.0
8,9,33.0,38.0,5.0
9,10,48.0,53.0,5.0
0,1,49.0,53.0,4.0
2,3,36.0,40.0,4.0
6,7,43.0,46.0,3.0
7,8,29.0,31.0,2.0
11,12,33.0,35.0,2.0
3,4,25.0,25.0,0.0
10,11,57.0,56.0,-1.0


In [6]:
r=round(q.groupby('ad')[['erro_mm','erro_s_mm']].mean()/10**6).reset_index()
r['dif']=r.erro_s_mm-r.erro_mm
r.sort_values(by='dif', ascending=False)

,ad,erro_mm,erro_s_mm,dif
6,7,27.0,35.0,8.0
5,6,42.0,44.0,2.0
7,8,39.0,41.0,2.0
4,5,42.0,40.0,-2.0
0,1,53.0,50.0,-3.0
3,4,57.0,53.0,-4.0
2,3,75.0,56.0,-19.0
1,2,37.0,8.0,-29.0


In [7]:
r=round(q.groupby('train_size_anos')[['erro_mm','erro_s_mm']].mean()/10**6).reset_index()
r['dif']=r.erro_s_mm-r.erro_mm
r.sort_values(by='dif', ascending=False)

,train_size_anos,erro_mm,erro_s_mm,dif
3,8.0,37.0,41.0,4.0
5,10.0,39.0,42.0,3.0
7,12.0,42.0,45.0,3.0
8,13.0,44.0,47.0,3.0
9,14.0,43.0,46.0,3.0
1,6.0,36.0,38.0,2.0
2,7.0,37.0,39.0,2.0
4,9.0,39.0,41.0,2.0
0,5.0,34.0,35.0,1.0
6,11.0,42.0,43.0,1.0


In [8]:
q.to_csv('~/git/Pricing-Fertilizer-take-Rains-into-account/temp/q.csv', index=False)